In [7]:
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows"

import sys
import os
import pandas as pd
from datetime import datetime

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule_weather import generate_activity_list, schedule_activities, scheduled_df, shift_start_end
from plot import plot

In [8]:
#read in activity data from excel
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Sheet1", skiprows=1)
activities = generate_activity_list(act_df)

In [6]:

# Load weather/tide data
weather_df = pd.read_csv(
    r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows\scripts\combined_tide_data_with_rate.csv"
)

 
# Parse Time column
weather_df['Time'] = pd.to_datetime(weather_df['Time'])

# Ensure absolute rate for current proxy
weather_df['Tide_rate_m_per_hr_abs'] = weather_df['Tide_rate_m_per_hr'].abs()


weather_data = pd.DataFrame({
    'datetime': weather_df['Time'],
    'tidal_level': weather_df['Tide Prediction (m)'],
    'tidal_current': weather_df['Tide_rate_m_per_hr_abs']
})


weather_data.head()

,datetime,tidal_level,tidal_current
0,2025-10-27 00:00:00,2.85,0.00
1,2025-10-27 00:10:00,2.92,0.42
2,2025-10-27 00:20:00,3.00,0.48
3,2025-10-27 00:30:00,3.07,0.42
4,2025-10-27 00:40:00,3.15,0.48


In [ ]:

# Load your activity DataFrame (act_df) from CSV or other source
activities = generate_activity_list(act_df)
scheduled = schedule_activities(activities, weather_data, analysis_interval=1.0, start_datetime=start_time)
df = scheduled_df(scheduled)